# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=Classes.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 Classes

This notebook contains high-level classes to interact with Mango Markets.

These often depend on structure layouts in [Layouts](Layouts.ipynb).

The idea is to have one high-level class, with useful members and methods, that can allow the Solana blobs loaded by the layout to vary. So there could be a layouts.SOMETHING_V1 and a layouts.SOMETHING_V2, and the Something class here can load from both.

Code, in general, depends only on these classes, abstracting away the version of the layout used to load the data.


In [ ]:
import datetime
import enum
import typing

import Layouts as layouts

from decimal import Decimal
from pyserum.market import Market
from solana.publickey import PublicKey
from solana.rpc.commitment import Single
from solana.rpc.types import MemcmpOpts

from Constants import NUM_MARKETS, NUM_TOKENS, SYSTEM_PROGRAM_ADDRESS
from Context import AccountInfo, Context
from Decoder import encode_key


## Version enum

This is used to keep track of which version of the layout struct was used to load the data.


In [ ]:
class Version(enum.Enum):
    UNSPECIFIED = 0
    V1 = 1
    V2 = 2
    V3 = 3
    V4 = 4
    V5 = 5


## SerumAccountFlags class

The Serum prefix is because there's also `MangoAccountFlags` for the Mango-specific flags.

In [ ]:
class SerumAccountFlags:
    def __init__(self, version: Version, initialized: bool, market: bool, open_orders: bool, request_queue: bool,
                event_queue: bool, bids: bool, asks: bool, disabled: bool):
        self.version: Version = version
        self.initialized = initialized
        self.market = market
        self.open_orders = open_orders
        self.request_queue = request_queue
        self.event_queue = event_queue
        self.bids = bids
        self.asks = asks
        self.disabled = disabled

    @staticmethod
    def from_layout(layout: layouts.SERUM_ACCOUNT_FLAGS) -> "SerumAccountFlags":
        return SerumAccountFlags(Version.UNSPECIFIED, layout.initialized, layout.market, layout.open_orders,
                            layout.request_queue, layout.event_queue, layout.bids, layout.asks, layout.disabled)

    def __str__(self) -> str:
        flags = []
        if self.initialized: flags += ["initialized"]
        if self.market: flags += ["market"]
        if self.open_orders: flags += ["open_orders"]
        if self.request_queue: flags += ["request_queue"]
        if self.event_queue: flags += ["event_queue"]
        if self.bids: flags += ["bids"]
        if self.asks: flags += ["asks"]
        if self.disabled: flags += ["disabled"]

        flag_text = " | ".join(flags) or "None"
        return f"« SerumAccountFlags: {flag_text} »"

    def __repr__(self) -> str:
        return f"{self}"


## MangoAccountFlags class

The Mango prefix is because there's also `SerumAccountFlags` for the standard Serum flags.

In [ ]:
class MangoAccountFlags:
    def __init__(self, version: Version, initialized: bool, group: bool, margin_account: bool, srm_account: bool):
        self.version: Version = version
        self.initialized = initialized
        self.group = group
        self.margin_account = margin_account
        self.srm_account = srm_account

    @staticmethod
    def from_layout(layout: layouts.MANGO_ACCOUNT_FLAGS) -> "MangoAccountFlags":
        return MangoAccountFlags(Version.UNSPECIFIED, layout.initialized, layout.group, layout.margin_account,
                                 layout.srm_account)

    def __str__(self) -> str:
        flags = []
        if self.initialized: flags += ["initialized"]
        if self.group: flags += ["group"]
        if self.margin_account: flags += ["margin_account"]
        if self.srm_account: flags += ["srm_account"]

        flag_text = " | ".join(flags) or "None"
        return f"« MangoAccountFlags: {flag_text} »"

    def __repr__(self) -> str:
        return f"{self}"


## Index class

In [ ]:
class Index:
    def __init__(self, version: Version, last_update: datetime.datetime, borrow: Decimal, deposit: Decimal):
        self.version: Version = version
        self.last_update: datetime.datetime = last_update
        self.borrow: Decimal = borrow
        self.deposit: Decimal = deposit

    @staticmethod
    def from_layout(layout: layouts.INDEX, decimals: Decimal) -> "Index":
        borrow = layout.borrow / Decimal(10 ** decimals)
        deposit = layout.deposit / Decimal(10 ** decimals)
        return Index(Version.UNSPECIFIED, layout.last_update, borrow, deposit)

    def __str__(self) -> str:
        return f"« Index: Borrow: {self.borrow:,.8f}, Deposit: {self.deposit:,.8f} [last update: {self.last_update}] »"

    def __repr__(self) -> str:
        return f"{self}"


## AggregatorConfig class

In [ ]:
class AggregatorConfig:
    def __init__(self, version: Version, description: str, decimals: Decimal, restart_delay: Decimal,
                 max_submissions: Decimal, min_submissions: Decimal, reward_amount: Decimal,
                 reward_token_account: PublicKey):
        self.version: Version = version
        self.description: str = description
        self.decimals: Decimal = decimals
        self.restart_delay: Decimal = restart_delay
        self.max_submissions: Decimal = max_submissions
        self.min_submissions: Decimal = min_submissions
        self.reward_amount: Decimal = reward_amount
        self.reward_token_account: PublicKey = reward_token_account

    @staticmethod
    def from_layout(layout: layouts.AGGREGATOR_CONFIG) -> "AggregatorConfig":
        return AggregatorConfig(Version.UNSPECIFIED, layout.description, layout.decimals,
                                layout.restart_delay, layout.max_submissions, layout.min_submissions,
                                layout.reward_amount, layout.reward_token_account)

    def __str__(self) -> str:
        return f"« AggregatorConfig: '{self.description}', Decimals: {self.decimals} [restart delay: {self.restart_delay}], Max: {self.max_submissions}, Min: {self.min_submissions}, Reward: {self.reward_amount}, Reward Account: {self.reward_token_account} »"

    def __repr__(self) -> str:
        return f"{self}"


## Round class

In [ ]:
class Round:
    def __init__(self, version: Version, id: Decimal, created_at: datetime.datetime, updated_at: datetime.datetime):
        self.version: Version = version
        self.id: Decimal = id
        self.created_at: datetime.datetime = created_at
        self.updated_at: datetime.datetime = updated_at

    @staticmethod
    def from_layout(layout: layouts.ROUND) -> "Round":
        return Round(Version.UNSPECIFIED, layout.id, layout.created_at, layout.updated_at)

    def __str__(self) -> str:
        return f"« Round[{self.id}], Created: {self.updated_at}, Updated: {self.updated_at} »"

    def __repr__(self) -> str:
        return f"{self}"


## Answer class

In [ ]:
class Answer:
    def __init__(self, version: Version, round_id: Decimal, median: Decimal, created_at: datetime.datetime, updated_at: datetime.datetime):
        self.version: Version = version
        self.round_id: Decimal = round_id
        self.median: Decimal = median
        self.created_at: datetime.datetime = created_at
        self.updated_at: datetime.datetime = updated_at

    @staticmethod
    def from_layout(layout: layouts.ANSWER) -> "Answer":
        return Answer(Version.UNSPECIFIED, layout.round_id, layout.median, layout.created_at, layout.updated_at)

    def __str__(self) -> str:
        return f"« Answer: Round[{self.round_id}], Median: {self.median:,.8f}, Created: {self.updated_at}, Updated: {self.updated_at} »"

    def __repr__(self) -> str:
        return f"{self}"


## Aggregator class

In [ ]:
class Aggregator:
    def __init__(self, version: Version, config: AggregatorConfig, initialized: bool, name: str,
                 address: PublicKey, owner: PublicKey, round_: Round, round_submissions: PublicKey,
                 answer: Answer, answer_submissions: PublicKey):
        self.version: Version = version
        self.config: AggregatorConfig = config
        self.initialized: bool = initialized
        self.name: str = name
        self.address: PublicKey = address
        self.owner: PublicKey = owner
        self.round: Round = round_
        self.round_submissions: PublicKey = round_submissions
        self.answer: Answer = answer
        self.answer_submissions: PublicKey = answer_submissions

    @property
    def price(self) -> Decimal:
        return self.answer.median / (10 ** self.config.decimals)

    @staticmethod
    def from_layout(layout: layouts.AGGREGATOR, context: Context, address: PublicKey) -> "Aggregator":
        config = AggregatorConfig.from_layout(layout.config)
        initialized = bool(layout.initialized)
        name = context.lookup_oracle_name(address)
        round_ = Round.from_layout(layout.round)
        answer = Answer.from_layout(layout.answer)

        return Aggregator(Version.UNSPECIFIED, config, initialized, name, address, layout.owner, round_,
                          layout.round_submissions, answer, layout.answer_submissions)

    @staticmethod
    def parse(data: bytes, context: Context, address: PublicKey) -> "Aggregator":
        if len(data) != layouts.AGGREGATOR.sizeof():
            raise Exception(f"Data length ({len(data)}) does not match expected size ({layouts.AGGREGATOR.sizeof()})")

        layout = layouts.AGGREGATOR.parse(data)
        return Aggregator.from_layout(layout, context, address)

    @staticmethod
    def load(context: Context, account_address: PublicKey):
        aggregator_result = context.load_account(account_address)
        return Aggregator.parse(aggregator_result.data, context, account_address)

    def __str__(self) -> str:
        return f"""
« Aggregator '{self.name}' [{self.version}]:
    Config: {self.config}
    Initialized: {self.initialized}
    Owner: {self.owner}
    Round: {self.round}
    Round Submissions: {self.round_submissions}
    Answer: {self.answer}
    Answer Submissions: {self.answer_submissions}
»
"""

    def __repr__(self) -> str:
        return f"{self}"


## BasketTokenMetadata class


In [ ]:
class BasketTokenMetadata:
    def __init__(self, name: str, mint: PublicKey, decimals: Decimal, vault: PublicKey, index: Index):
        self.name: str = name
        self.mint: PublicKey = mint
        self.decimals: Decimal = decimals
        self.vault: PublicKey = vault
        self.index: Index = index

    def round(self, value: Decimal) -> Decimal:
        rounded = round(value, int(self.decimals))
        return Decimal(rounded)

    def __str__(self) -> str:
        return f"""« Token '{self.name}' [{self.mint} ({self.decimals} decimals)]:
    Vault: {self.vault}
    Index: {self.index}
»"""

    def __repr__(self) -> str:
        return f"{self}"


## MarketMetadata class

In [ ]:
class MarketMetadata:
    def __init__(self, name: str, address: PublicKey, base: BasketTokenMetadata, quote: BasketTokenMetadata,
                 spot: PublicKey, oracle: PublicKey, decimals: Decimal):
        self.name: str = name
        self.address: PublicKey = address
        self.base: BasketTokenMetadata = base
        self.quote: BasketTokenMetadata = quote
        self.spot: PublicKey = spot
        self.oracle: PublicKey = oracle
        self.decimals: Decimal = decimals
        self._market = None

    def fetch_market(self, context: Context) -> Market:
        if self._market is None:
            self._market = Market.load(context.client, self.spot)

        return self._market

    def __str__(self) -> str:
        return f"""« Market '{self.name}' [{self.spot}]:
    Base: {self.base}
    Quote: {self.quote}
    Oracle: {self.oracle} ({self.decimals} decimals)
»"""

    def __repr__(self) -> str:
        return f"{self}"


## Group class

In [ ]:
class Group:
    def __init__(self, version: Version, context: Context, address: PublicKey,
                 account_flags: MangoAccountFlags, tokens: typing.List[BasketTokenMetadata],
                 markets: typing.List[MarketMetadata],
                 signer_nonce: Decimal, signer_key: PublicKey, dex_program_id: PublicKey,
                 total_deposits: typing.List[Decimal], total_borrows: typing.List[Decimal],
                 maint_coll_ratio: Decimal, init_coll_ratio: Decimal, srm_vault: PublicKey,
                 admin: PublicKey, borrow_limits: typing.List[Decimal]):
        self.version: Version = version
        self.context: Context = context
        self.address: PublicKey = address
        self.account_flags: MangoAccountFlags = account_flags
        self.tokens: typing.List[BasketTokenMetadata] = tokens
        self.markets: typing.List[MarketMetadata] = markets
        self.signer_nonce: Decimal = signer_nonce
        self.signer_key: PublicKey = signer_key
        self.dex_program_id: PublicKey = dex_program_id
        self.total_deposits: typing.List[Decimal] = total_deposits
        self.total_borrows: typing.List[Decimal] = total_borrows
        self.maint_coll_ratio: Decimal = maint_coll_ratio
        self.init_coll_ratio: Decimal = init_coll_ratio
        self.srm_vault: PublicKey = srm_vault
        self.admin: PublicKey = admin
        self.borrow_limits: typing.List[Decimal] = borrow_limits

    @property
    def shared_quote_token(self) -> BasketTokenMetadata:
        return self.tokens[NUM_TOKENS - 1]

    @staticmethod
    def from_layout(layout: layouts.GROUP, context: Context) -> "Group":
        account_flags = MangoAccountFlags.from_layout(layout.account_flags)
        indexes = list(map(lambda pair: Index.from_layout(pair[0], pair[1]), zip(layout.indexes, layout.mint_decimals)))

        tokens: typing.List[BasketTokenMetadata] = []
        for index in range(NUM_TOKENS):
            token_address = layout.tokens[index]
            token_name = context.lookup_token_name(token_address)
            token = BasketTokenMetadata(token_name, token_address,
                                        layout.mint_decimals[index],
                                        layout.vaults[index],
                                        indexes[index])
            tokens += [token]

        markets: typing.List[MarketMetadata] = []
        for index in range(NUM_MARKETS):
            market_address = layout.spot_markets[index]
            market_name = context.lookup_market_name(market_address)
            base_name, quote_name = market_name.split("/")
            base_token = [token for token in tokens if token.name == base_name][0]
            quote_token = [token for token in tokens if token.name == quote_name][0]
            market = MarketMetadata(market_name, market_address, base_token, quote_token,
                                    layout.spot_markets[index],
                                    layout.oracles[index],
                                    layout.oracle_decimals[index])
            markets += [market]

        maint_coll_ratio = layout.maint_coll_ratio.quantize(Decimal('.01'))
        init_coll_ratio = layout.init_coll_ratio.quantize(Decimal('.01'))
        return Group(Version.UNSPECIFIED, context, context.group_id, account_flags, tokens, markets,
                     layout.signer_nonce, layout.signer_key, layout.dex_program_id, layout.total_deposits,
                     layout.total_borrows, maint_coll_ratio, init_coll_ratio, layout.srm_vault,
                     layout.admin, layout.borrow_limits)

    @staticmethod
    def parse(data: bytes, context: Context) -> "Group":
        if len(data) != layouts.GROUP.sizeof():
            raise Exception(f"Data length ({len(data)}) does not match expected size ({layouts.GROUP.sizeof()})")

        layout = layouts.GROUP.parse(data)
        return Group.from_layout(layout, context)

    @staticmethod
    def load(context: Context):
        group_account_info = context.load_account(context.group_id);
        return Group.parse(group_account_info.data, context)

    def get_prices(self):
        oracles = map(lambda market: Aggregator.load(self.context, market.oracle), self.markets)
        return list(map(lambda oracle: oracle.price, oracles)) + [Decimal(1)]

    def __str__(self) -> str:
        total_deposits = "\n        ".join(map(str, self.total_deposits))
        total_borrows = "\n        ".join(map(str, self.total_borrows))
        borrow_limits = "\n        ".join(map(str, self.borrow_limits))
        return f"""
« Group [{self.version}] {self.address}:
    Flags: {self.account_flags}
    Tokens:
{self.tokens}
    Markets:
{self.markets}
    DEX Program ID: « {self.dex_program_id} »
    SRM Vault: « {self.srm_vault} »
    Admin: « {self.admin} »
    Signer Nonce: {self.signer_nonce}
    Signer Key: « {self.signer_key} »
    Initial Collateral Ratio: {self.init_coll_ratio}
    Maintenance Collateral Ratio: {self.maint_coll_ratio}
    Total Deposits:
        {total_deposits}
    Total Borrows:
        {total_borrows}
    Borrow Limits:
        {borrow_limits}
»
"""

    def __repr__(self) -> str:
        return f"{self}"


## TokenAccount class

In [ ]:
class TokenAccount:
    def __init__(self, version: Version, mint: PublicKey, owner: PublicKey, amount: Decimal):
        self.version: Version = version
        self.mint: PublicKey = mint
        self.owner: PublicKey = owner
        self.amount: Decimal = amount

    @staticmethod
    def from_layout(layout: layouts.TOKEN_ACCOUNT) -> "TokenAccount":
        return TokenAccount(Version.UNSPECIFIED, layout.mint, layout.owner, layout.amount)

    @staticmethod
    def parse(data) -> "TokenAccount":
        if len(data) != layouts.TOKEN_ACCOUNT.sizeof():
            raise Exception(f"Data length ({len(data)}) does not match expected size ({layouts.TOKEN_ACCOUNT.sizeof()})")

        layout = layouts.TOKEN_ACCOUNT.parse(data)
        return TokenAccount.from_layout(layout)

    def __str__(self) -> str:
        return f"« Token: Mint: {self.mint}, Owner: {self.owner}, Amount: {self.amount} »"

    def __repr__(self) -> str:
        return f"{self}"


## OpenOrders class


In [ ]:
class OpenOrders:
    def __init__(self, version: Version, group: Group, address: PublicKey, program_id: PublicKey,
                 account_flags: SerumAccountFlags, market: PublicKey, owner: PublicKey,
                 base_token_free: Decimal, base_token_total: Decimal, quote_token_free: Decimal,
                 quote_token_total: Decimal, free_slot_bits: Decimal, is_bid_bits: Decimal,
                 orders: typing.List[Decimal], client_ids: typing.List[Decimal],
                 referrer_rebate_accrued: Decimal):
        self.version: Version = version
        self.group: Group = group
        self.address: PublicKey = address
        self.program_id: PublicKey = program_id
        self.account_flags: SerumAccountFlags = account_flags
        self.market: PublicKey = market
        self.owner: PublicKey = owner
        self.base_token_free: Decimal = base_token_free
        self.base_token_total: Decimal = base_token_total
        self.quote_token_free: Decimal = quote_token_free
        self.quote_token_total: Decimal = quote_token_total
        self.free_slot_bits: Decimal = free_slot_bits
        self.is_bid_bits: Decimal = is_bid_bits
        self.orders: typing.List[Decimal] = orders
        self.client_ids: typing.List[Decimal] = client_ids
        self.referrer_rebate_accrued: Decimal = referrer_rebate_accrued

    @staticmethod
    def from_layout(layout: layouts.OPEN_ORDERS, address: PublicKey, program_id: PublicKey,
                    group: Group, base_decimals: Decimal, quote_decimals: Decimal) -> "OpenOrders":
        account_flags = SerumAccountFlags.from_layout(layout.account_flags)

        base_divisor = 10 ** base_decimals
        quote_divisor = 10 ** quote_decimals
        base_token_free: Decimal = layout.base_token_free / base_divisor
        base_token_total: Decimal = layout.base_token_total / base_divisor
        quote_token_free: Decimal = layout.quote_token_free / quote_divisor
        quote_token_total: Decimal = layout.quote_token_total / quote_divisor
        nonzero_orders: typing.List[Decimal] = list([order for order in layout.orders if order != 0])
        nonzero_client_ids: typing.List[Decimal] = list([client_id for client_id in layout.client_ids if client_id != 0])

        return OpenOrders(Version.UNSPECIFIED, group, address, program_id, account_flags, layout.market,
                          layout.owner, base_token_free, base_token_total, quote_token_free, quote_token_total,
                          layout.free_slot_bits, layout.is_bid_bits, nonzero_orders, nonzero_client_ids,
                          layout.referrer_rebate_accrued)

    @staticmethod
    def parse(data: bytes, address: PublicKey, program_id: PublicKey, group: Group, base_decimals: Decimal, quote_decimals: Decimal) -> "OpenOrders":
        if len(data) != layouts.OPEN_ORDERS.sizeof():
            raise Exception(f"Data length ({len(data)}) does not match expected size ({layouts.OPEN_ORDERS.sizeof()})")

        layout = layouts.OPEN_ORDERS.parse(data)
        return OpenOrders.from_layout(layout, address, program_id, group, base_decimals, quote_decimals)

    @staticmethod
    def load_raw_open_orders_accounts(context: Context, group: Group) -> typing.List[typing.Tuple[PublicKey, typing.Any]]: # Return type should be typing.Tuple[PublicKey, layouts.OPEN_ORDERS]
        filters = [
            MemcmpOpts(
                offset=layouts.SERUM_ACCOUNT_FLAGS.sizeof() + 37,
                bytes=encode_key(group.signer_key)
            )
        ]

        response = context.client.get_program_accounts(group.dex_program_id, data_size=layouts.OPEN_ORDERS.sizeof(), memcmp_opts=filters, commitment=Single, encoding="base64")
        accounts = list(map(lambda pair: AccountInfo._from_response_values(pair[0], pair[1]), [(result["account"], PublicKey(result["pubkey"])) for result in response["result"]]))
        structs = list(map(lambda acc: (acc.address, layouts.OPEN_ORDERS.parse(acc.data)), accounts))
        return structs

    @staticmethod
    def load(context: Context, address: PublicKey, group: Group, base_decimals: Decimal, quote_decimals: Decimal):
        acc = context.load_account(address)
        return OpenOrders.parse(acc.data, acc.address, acc.owner, group, base_decimals, quote_decimals)


    def __str__(self) -> str:
        orders = ", ".join(map(str, self.orders)) or "None"
        client_ids = ", ".join(map(str, self.client_ids)) or "None"
        
        return f"""« OpenOrders:
    Flags: {self.account_flags}
    Program ID: {self.program_id}
    Address: {self.address}
    Market: {self.market}
    Owner: {self.owner}
    Base Token: {self.base_token_free:,.8f} of {self.base_token_total:,.8f}
    Quote Token: {self.quote_token_free:,.8f} of {self.quote_token_total:,.8f}
    Referrer Rebate Accrued: {self.referrer_rebate_accrued}
    Orders:
        {orders}
    Client IDs:
        {client_ids}
»"""

    def __repr__(self) -> str:
        return f"{self}"


## BalanceSheet class

In [ ]:
class BalanceSheet:
    def __init__(self, liabilities: Decimal, settled_assets: Decimal, unsettled_assets: Decimal):
        self.liabilities: Decimal = liabilities
        self.settled_assets: Decimal = settled_assets
        self.unsettled_assets: Decimal = unsettled_assets

    @property
    def assets(self) -> Decimal:
        return self.settled_assets + self.unsettled_assets

    @property
    def value(self) -> Decimal:
        return self.assets - self.liabilities

    @property
    def collateral_ratio(self) -> Decimal:
        if self.liabilities == Decimal(0):
            return Decimal(0)
        return self.assets / self.liabilities

    def __str__(self) -> str:
        return f"""« BalanceSheet:
    Assets :           {self.assets:>18,.8f}
    Settled Assets :   {self.settled_assets:>18,.8f}
    Unsettled Assets : {self.unsettled_assets:>18,.8f}
    Liabilities :      {self.liabilities:>18,.8f}
    Value :            {self.value:>18,.8f}
    Collateral Ratio : {self.collateral_ratio:>18,.2%}
»
"""

    def __repr__(self) -> str:
        return f"{self}"


## MarginAccount class


In [ ]:
class MarginAccount:
    def __init__(self, version: Version, address: PublicKey, account_flags: MangoAccountFlags,
                 mango_group: PublicKey, owner: PublicKey, deposits: typing.List[Decimal],
                 borrows: typing.List[Decimal], open_orders: typing.List[PublicKey]):
        self.version: Version = version
        self.address: PublicKey = address
        self.account_flags: MangoAccountFlags = account_flags
        self.mango_group: PublicKey = mango_group
        self.owner: PublicKey = owner
        self.deposits: typing.List[Decimal] = deposits
        self.borrows: typing.List[Decimal] = borrows
        self.open_orders: typing.List[PublicKey] = open_orders
        self.open_orders_accounts: typing.List[typing.Optional[OpenOrders]] = [None] * NUM_MARKETS

    @staticmethod
    def from_layout(layout: layouts.MARGIN_ACCOUNT, address: PublicKey) -> "MarginAccount":
        account_flags: MangoAccountFlags = MangoAccountFlags.from_layout(layout.account_flags)
        deposits: typing.List[Decimal] = []
        for index, deposit in enumerate(layout.deposits):
            deposits += [deposit]

        borrows: typing.List[Decimal] = []
        for index, borrow in enumerate(layout.borrows):
            borrows += [borrow]

        return MarginAccount(Version.UNSPECIFIED, address, account_flags, layout.mango_group,
                             layout.owner, deposits, borrows, list(layout.open_orders))

    @staticmethod
    def parse(data: bytes, address: PublicKey) -> "MarginAccount":
        if len(data) != layouts.MARGIN_ACCOUNT.sizeof():
            raise Exception(f"Data length ({len(data)}) does not match expected size ({layouts.MARGIN_ACCOUNT.sizeof()})")

        layout = layouts.MARGIN_ACCOUNT.parse(data)
        return MarginAccount.from_layout(layout, address)

    @staticmethod
    def load(context: Context, group: Group, margin_account_address: PublicKey) -> "MarginAccount":
        account = context.load_account(margin_account_address)
        margin_account = MarginAccount.parse(account.data, margin_account_address)
        margin_account.load_open_orders_accounts(context, group)
        return margin_account

    @staticmethod
    def load_all_for_group(context: Context, program_id: PublicKey, group: Group) -> typing.List["MarginAccount"]:
        filters = [
            MemcmpOpts(
                offset=layouts.MANGO_ACCOUNT_FLAGS.sizeof(), # mango_group is just after the MangoAccountFlags, which is the first entry
                bytes=encode_key(group.address)
            )
        ]
        response = context.client.get_program_accounts(program_id, data_size=layouts.MARGIN_ACCOUNT.sizeof(), memcmp_opts=filters, commitment=Single, encoding="base64")
        margin_accounts = []
        for margin_account_data in response["result"]:
            address = PublicKey(margin_account_data["pubkey"])
            account = AccountInfo._from_response_values(margin_account_data["account"], address)
            margin_account = MarginAccount.parse(account.data, address)
            margin_accounts += [margin_account]
        return margin_accounts

    @staticmethod
    def load_all_for_owner(context: Context, program_id: PublicKey, group: Group, owner: PublicKey) -> typing.List["MarginAccount"]:
        mango_group_offset = layouts.MANGO_ACCOUNT_FLAGS.sizeof() # mango_group is just after the MangoAccountFlags, which is the first entry.
        owner_offset = mango_group_offset + 32 # owner is just after mango_group in the layout, and it's a PublicKey which is 32 bytes.
        filters = [
            MemcmpOpts(
                offset=mango_group_offset,
                bytes=encode_key(group.address)
            ),
            MemcmpOpts(
                offset=owner_offset,
                bytes=encode_key(owner)
            )
        ]

        response = context.client.get_program_accounts(program_id, data_size=layouts.MARGIN_ACCOUNT.sizeof(), memcmp_opts=filters, commitment=Single, encoding="base64")
        margin_accounts = []
        for margin_account_data in response["result"]:
            address = PublicKey(margin_account_data["pubkey"])
            account = AccountInfo._from_response_values(margin_account_data["account"], address)
            margin_account = MarginAccount.parse(account.data, address)
            margin_account.load_open_orders_accounts(context, group)
            margin_accounts += [margin_account]
        return margin_accounts

    def load_open_orders_accounts(self, context: Context, group: Group) -> None:
        for index, oo in enumerate(self.open_orders):
            key = oo
            if key != SYSTEM_PROGRAM_ADDRESS:
                self.open_orders_accounts[index] = OpenOrders.load(context, key, group, group.tokens[index].decimals, group.shared_quote_token.decimals)

    def install_open_orders_accounts(self, group: Group, all_open_orders_by_address: typing.Dict[str, typing.Any]) -> None: # Dict value type should be layouts.OPEN_ORDERS
        for index, oo in enumerate(self.open_orders):
            key = str(oo)
            if key in all_open_orders_by_address:
                open_orders_layout = all_open_orders_by_address[key]
                open_orders = OpenOrders.from_layout(open_orders_layout, oo, group.context.program_id,
                                                     group, group.tokens[index].decimals,
                                                     group.shared_quote_token.decimals)
                self.open_orders_accounts[index] = open_orders

    def get_native_balance_sheets(self, group: Group) -> typing.List[typing.Optional[BalanceSheet]]:
        settled_assets: typing.List[Decimal] = [Decimal(0)] * NUM_TOKENS
        liabilities: typing.List[Decimal] = [Decimal(0)] * NUM_TOKENS
        for index in range(NUM_TOKENS):
            settled_assets[index] = group.tokens[index].index.deposit * self.deposits[index]
            liabilities[index] = group.tokens[index].index.borrow * self.borrows[index]

        unsettled_assets: typing.List[Decimal] = [Decimal(0)] * NUM_TOKENS
        for index in range(NUM_MARKETS):
            open_orders_account = self.open_orders_accounts[index]
            if open_orders_account is not None:
                unsettled_assets[index] = open_orders_account.base_token_total
                unsettled_assets[NUM_TOKENS - 1] = open_orders_account.quote_token_total

        balance_sheets: typing.List[typing.Optional[BalanceSheet]] = [None] * NUM_TOKENS
        for index in range(NUM_TOKENS):
            balance_sheets[index] = BalanceSheet(liabilities[index], settled_assets[index], unsettled_assets[index])

        return balance_sheets

    def get_priced_balance_sheets(self, group: Group, prices: typing.List[Decimal]) -> typing.List[typing.Optional[BalanceSheet]]:
        priced: typing.List[typing.Optional[BalanceSheet]] = [None] * NUM_TOKENS
        balance_sheets = self.get_native_balance_sheets(group)
        for index, balance_sheet in enumerate(balance_sheets):
            if balance_sheet is not None:
                liabilities = balance_sheet.liabilities * prices[index]
                settled_assets = balance_sheet.settled_assets * prices[index]
                unsettled_assets = balance_sheet.unsettled_assets * prices[index]
                token_metadata = group.tokens[index]
                priced[index] = BalanceSheet(
                    token_metadata.round(liabilities),
                    token_metadata.round(settled_assets),
                    token_metadata.round(unsettled_assets)
                )

        return priced

    def get_balance_sheet_totals(self, group: Group, prices: typing.List[Decimal]) -> BalanceSheet:
        liabilities = Decimal(0)
        settled_assets = Decimal(0)
        unsettled_assets = Decimal(0)

        balance_sheets = self.get_priced_balance_sheets(group, prices)
        for balance_sheet in balance_sheets:
            if balance_sheet is not None:
                liabilities += balance_sheet.liabilities
                settled_assets += balance_sheet.settled_assets
                unsettled_assets += balance_sheet.unsettled_assets

        return BalanceSheet(liabilities, settled_assets, unsettled_assets)

    def __str__(self) -> str:
        deposits = ", ".join([f"{item:,.8f}" for item in self.deposits])
        borrows = ", ".join([f"{item:,.8f}" for item in self.borrows])
        if all(oo is None for oo in self.open_orders_accounts):
            open_orders = f"{self.open_orders}"
        else:
            open_orders_unindented = f"{self.open_orders_accounts}"
            open_orders = open_orders_unindented.replace("\n", "\n    ")
        return f"""« MarginAccount: {self.address}
    Flags: {self.account_flags}
    Owner: {self.owner}
    Mango Group: {self.mango_group}
    Deposits: [{deposits}]
    Borrows: [{borrows}]
    Open Orders: {open_orders}
»
"""

    def __repr__(self) -> str:
        return f"{self}"


# 🏃 Running

In [ ]:
if __name__ == "__main__":
    import base64
    from Context import default_context

    encoded = "AwAAAAAAAACCaOmpoURMK6XHelGTaFawcuQ/78/15LAemWI8jrt3SRKLy2R9i60eclDjuDS8+p/ZhvTUd9G7uQVOYCsR6+BhmqGCiO6EPYP2PQkf/VRTvw7JjXvIjPFJy06QR1Cq1WfTonHl0OjCkyEf60SD07+MFJu5pVWNFGGEO/8AiAYfduaKdnFTaZEHPcK5Eq72WWHeHg2yIbBF09kyeOhlCJwOoG8O5SgpPV8QOA64ZNV4aKroFfADg6kEy/wWCdp3fv0O4GJgAAAAAPH6Ud6jtjwAAQAAAAAAAADiDkkCi9UOAAEAAAAAAAAADuBiYAAAAACNS5bSy7soAAEAAAAAAAAACMvgO+2jCwABAAAAAAAAAA7gYmAAAAAAZFeDUBNVhwABAAAAAAAAABtRNytozC8AAQAAAAAAAABIBGiCcyaEZdNhrTyeqUY692vOzzPdHaxAxguht3JQGlkzjtd05dX9LENHkl2z1XvUbTNKZlweypNRetmH0lmQ9VYQAHqylxZVK65gEg85g27YuSyvOBZAjJyRmYU9KdCO1D+4ehdPu9dQB1yI1uh75wShdAaFn2o4qrMYwq3SQQEAAAAAAAAAAiH1PPJKAuh6oGiE35aGhUQhFi/bxgKOudpFv8HEHNCFDy1uAqR6+CTQmradxC1wyyjL+iSft+5XudJWwSdi7wvphsxb96x7Obj/AgAAAAAKlV4LL5ow6r9LMhIAAAAADvsOtqcVFmChDPzPnwAAAE33lx1h8hPFD04AAAAAAAA8YRV3Oa309B2wGwAAAAAA+yPBZRlZz7b605n+AQAAAACgmZmZmZkZAQAAAAAAAAAAMDMzMzMzMwEAAAAAAAAA25D1XcAtRzSuuyx3U+X7aE9vM1EJySU9KprgL0LMJ/vat9+SEEUZuga7O5tTUrcMDYWDg+LYaAWhSQiN2fYk7aCGAQAAAAAAgIQeAAAAAAAA8gUqAQAAAAYGBgICAAAA"
    decoded = base64.b64decode(encoded)

    group = Group.parse(decoded, default_context)
    print("\n\nThis is hard-coded, not live information!")
    print(group)
